<font size="1">Copyright 2021, by the California Institute of Technology. ALL RIGHTS RESERVED. United States Government sponsorship acknowledged. Any commercial use must be negotiated with the Office of Technology Transfer at the California Institute of Technology.</font>
    
<font size="1">This software may be subject to U.S. export control laws and regulations. By accepting this document, the user agrees to comply with all applicable U.S. export laws and regulations. User has the responsibility to obtain export licenses, or other export authority as may be required, before exporting such information to foreign countries or providing access to foreign persons.<font>

# Cloud-Scaling TopsApp Processing in the On-Demand NISAR Science Data System (SDS)

## Introduction
With TopsApp.ipynb, we can run topsApp in a notebook. Once topsApp.ipynb is integrated into the on-demand NISAR SDS, we can execute the topsApp notebook as a job in the on-demand NISAR SDS and run them at scale.    

In this notebook we will show, how to:
 - **Create input data for MULTIPLE topsApp processing**
 - **Get information about topsApp job**
 - **Submit MULTIPLE topsApp jobs SIMULTANEOUSLY with different inputs**
 - **Download generated products from the cloud**
 - **Plot the generated innformation**




## Configurable Parameters

We can run this notebook to process topsApp with many different combinations of SLCs and region of interest as well as with different combination of topsApp properties like swaths, azimuth looks, range looks etc. In the followinng section we initizale all these tunable variables.

**Input Parameters**

- **min_lat, max_lat, min_loc, max_loc**: min/max latitude and longditude of Region of Interest.

    Default values:
        min_lat = 31.9 # type: number
        max_lat = 33.94 # type: number
        min_lon =  -118.74 # type: number
        max_lon = -115.69 # type: number
       
- **reference_slcs, secondary_slcs** : List of reference SLCs and secondary SLCs.

    Default value:
        reference_slcs: List[str] = ["S1B_IW_SLC__1SDV_20190628T014909_20190628T014936_016890_01FC87_55C8"]
        secondary_slcs: List[str] = ["S1B_IW_SLC__1SDV_20190710T014909_20190710T014936_017065_0201B8_0252"]  
        
**Tunable Parameters**
- **swaths** : array conntaining swath values to be considerate.

    Default value:
        swaths: List[int] = [3]
- **range_looks** : range looks value. Number.

    Default value: 
        range_looks = 7        
- **azimuth_looks** : azimuth looks value. Number.

    Default value:
        azimuth_looks = 3
- **do_unwrap** : True or False if unwrapping shoud be done or not.

    Default value:
        do_unwrap = "True"
- **unwrapper_name** : Unwrapper name when do_unwrap is True.

    Default value:
        unwrapper_name = "snaphu_mcf"
- **do_denseoffsets** : True/False for denseoffsets processing.

    Default value:
        do_denseoffsets = "False"
           



In [ ]:

input_params = [ {
    "tags" : "test_from_jupyter-1",
    "params" : {  
        "min_lat" : 37.89,
        "max_lat" : 39.57,
        "min_lon" : -119.75,
        "max_lon" : -116.88,
        "reference_slcs" : ["S1B_IW_SLC__1SDV_20190628T015048_20190628T015116_016890_01FC87_50F1"],
        "secondary_slcs" : ["S1B_IW_SLC__1SDV_20190710T015049_20190710T015117_017065_0201B8_0114"]
         }
    },{
    "tags" : "test_from_jupyter-2",
    "params" : { 
        "min_lat" : 36.39,
        "max_lat" : 38.42,
        "min_lon" : -119.79,
        "max_lon" : -116.57,
        "reference_slcs" : ["S1B_IW_SLC__1SDV_20190628T015023_20190628T015050_016890_01FC87_AA6E"],
        "secondary_slcs" : ["S1B_IW_SLC__1SDV_20190710T015024_20190710T015051_017065_0201B8_ABFD"]
        }
    },{
    "tags" : "test_from_jupyter-3",
    "params" : {

        "min_lat" : 31.90,
        "max_lat" : 33.94,
        "min_lon" : -118.74,
        "max_lon" : -115.69,
        "reference_slcs" : ["S1B_IW_SLC__1SDV_20190628T014909_20190628T014936_016890_01FC87_55C8"],
        "secondary_slcs" : ["S1B_IW_SLC__1SDV_20190710T014909_20190710T014936_017065_0201B8_0252"]
        }
    },{
    "tags" : "test_from_jupyter-4",
    "params" : {

        "min_lat" : 33.40,
        "max_lat" : 35.44,
        "min_lon" : -119.09,
        "max_lon" : -115.98,
        "reference_slcs" : ["S1B_IW_SLC__1SDV_20190628T014933_20190628T015000_016890_01FC87_81EB"],
        "secondary_slcs" : ["S1B_IW_SLC__1SDV_20190710T014934_20190710T015001_017065_0201B8_4872"]
        }
    },{
    "tags" : "test_from_jupyter-5",
    "params" : {

        "min_lat" : 34.90,
        "max_lat" : 36.53,
        "min_lon" : -119.44,
        "max_lon" : -116.28,
        "reference_slcs" : ["S1B_IW_SLC__1SDV_20190628T014958_20190628T015025_016890_01FC87_FC0D"],
        "secondary_slcs" : ["S1B_IW_SLC__1SDV_20190710T014959_20190710T015026_017065_0201B8_069B"]
        }
    }
        
]

tunable_params = {
    "sensor_name" : "SENTINEL1",
    "swaths" : "[1, 2, 3]",
    "range_looks" : 7,
    "azimuth_looks" : 3,
    "do_unwrap" : "False",
    "unwrapper_name" : "snaphu_mcf",
    "do_denseoffsets" : "False"
}


## Setup configuration for interaction with on-demand NISAR SDS
In this section, we setup the necessary python libraries and API endpoints needed to interact with the **on-demand NISAR SDS**. 

The `otello` python library provides high-level access to operations on the on-demand SDS. In particular for this demo, it provides us with access to and information about the job types registered on the SDS along with the capability to submit jobs, check for job statuses, and to query for products generated by the job.

In [ ]:
import os, requests, json, getpass, time, re, shutil, sys
from pathlib import Path
from datetime import datetime
from requests.auth import HTTPBasicAuth
import urllib3
import otello
import os
from pprint import pprint

import os

# this block makes sure the directory set-up/change is only done once and relative to the notebook's directory
try:
    start_dir
except NameError:
    start_dir = os.getcwd()
    
output_dir = os.path.join(start_dir, 'notebook_output/topsApp')
python_dir = os.path.join(start_dir, 'python')

!mkdir -p $output_dir
os.chdir(output_dir)
sys.path.append(python_dir)

import plot_util


ISCE_HOME="/opt/isce2/isce"
urllib3.disable_warnings()

# set the mozart IP
mozart_ip = "100.64.122.59"
#mozart_ip = input("Enter IP address of your mozart instance then press <Enter>: ")
print(f"Using mozart IP address {mozart_ip}.")

if not os.path.exists(f"{Path.home()}/.config/otello/config.yml"):
    otello.client.initialize()

m = otello.Mozart()

## Submitting an "on-demand topsApp" job
### Next let's get more information on the topsApp Job:

In [ ]:
job_type = "job-topsApp:develop"
topsApp = m.get_job_types()[job_type]
topsApp.initialize()

### Prepare input for the MULTIPLE topsApp Jobs and submit them in bulk  to run in the on-demand NISAR SDS

Note that we are explicitly overriding the default job queue in the submit_job call to specify the (only) one available in the Beta PCM environment.

In [ ]:
job_set = otello.JobSet()
topsApp.set_input_params(tunable_params)
for i in range(len(input_params)):
    topsApp.set_input_params(input_params[i]["params"])
    job = topsApp.submit_job(tag=input_params[i]["tags"], queue="nisar-job_worker-small")
    job_set.append(job)

### Let's wait for the jobs to complete generation of the topsApp products

In [ ]:
job_set.wait_for_completion()

### Get information about the generated topsApp products from the topsApp jobs
#### The generated topsApp products are stored in the cloud next to the on-demand NISAR SDS

In [ ]:

prods = []
for job in job_set:
    try:
        prod = job.get_generated_products()
        print(json.dumps(prod, indent=2, sort_keys=True))
        prods.append(prod)
    except Exception as e:
        print(e)


### Download the generated standard topsApp products from the cloud into this notebook
#### Here we use the AWS CLI to download the generated datasets.

In [ ]:
local_dirs = []
for prod in prods:
    try:
        prod_url = re.sub(r'^s3://.+?/(.+)$', r's3://\1', prod[0]["urls"][-1]) # get s3 url
        local_dir = os.path.basename(prod_url)
        if os.path.isdir(local_dir): shutil.rmtree(local_dir)
        !aws s3 sync $prod_url $local_dir
        local_dirs.append(local_dir)
    except Exception as e:
        print(e)
for local_dir in local_dirs:
    !ls $local_dir

### Use matplotlib to visualize the topsApp products
Here we list the contents of the generated datasets. We plot filt_topophase.unw.geo file.

In [ ]:
import os
import folium
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import rasterio as rio
from rasterio.plot import show, plotting_extent
from rasterio.merge import merge
from PIL import Image, ImageChops


# plot wrapped IFGs individually
flat_plots = []
flat_bboxes = []
for i, file in enumerate(glob("hello_world*/filt_topophase.flat.geo")):
    src = rio.open(file)
    fig, ax = plt.subplots(1, figsize=(18, 16))
    data = src.read(1)
    data[data==0] = np.nan
    show(np.angle(data), cmap='rainbow', vmin=-np.pi, vmax=np.pi, transform=src.transform, ax=ax)
    png_file = f'flat_{i}.png'
    fig.savefig(png_file, transparent=True)
    flat_plots.append(png_file)
    flat_bboxes.append(src.bounds)

In [ ]:
# function to trim png images
def trim(img):
    border = Image.new(img.mode, img.size, img.getpixel((0, 0)))
    diff = ImageChops.difference(img, border)
    diff = ImageChops.add(diff, diff, 2.0, -100)
    bbox = diff.getbbox()
    if bbox:
        img = img.crop(bbox)
    return np.array(img)


# plot wrapped IFGs in folium map
m = folium.Map(location=[35.71975793933433, -117.8668212890625],
               tiles='Stamen Terrain', zoom_start=6)
for i, plot in enumerate(flat_plots):
    bounds = [[flat_bboxes[i].bottom, flat_bboxes[i].left],
              [flat_bboxes[i].top, flat_bboxes[i].right]]
    img = trim(Image.open(plot))
    o = folium.raster_layers.ImageOverlay(image=img,
                                          bounds=bounds,
                                          opacity=.7,
                                          interactive=True,
                                          cross_origin=False)
    o.add_to(m)
folium.LayerControl().add_to(m)             
m